# ML for Causal Inference: High-Dimensional Controls

In [11]:
library(simstudy)
library(glmnet)
library(rdd)

## Simulate Data

In [12]:
set.seed(1)
# Number of Observations
N <- 1e3

total.covar <- 50 + 1e3
# Number of covariates (excluding W and unobservable)
p <- total.covar - 2

mu.vector <- rep(0, total.covar)
variance.vector <- abs(rnorm(total.covar, mean = 1, sd = .5))
simulated.data <- as.data.frame.matrix(genCorGen(n = N, nvars = total.covar, params1 = mu.vector, params2 = variance.vector, dist = 'normal',  rho = .5,
                            corstr = 'ar1', wide='True'))[2:(total.covar+1)]
colnames(simulated.data)[1] <- 'W'
colnames(simulated.data)[total.covar] <- 'C'


X <- simulated.data[, 2:(total.covar-1)]

covariate.names <- colnames(X)

error <- rnorm(n = N)

# Make W a function of the X's and unobservable
simulated.data$W <- simulated.data$W + .5 * simulated.data$C + 3 * simulated.data$V80 - 6 * simulated.data$V81
# Assign treatment
treated <- (simulated.data$W > 0) * 1.0


# Generate Y as a function of treatment, W, X's, and unobservable C
beta.true.linear <- rnorm(p, mean = 5, sd = 5)
beta.true.linear[30:p] <- 0
Y <- 1.2 * treated - 4 * simulated.data$W  + data.matrix(X) %*% beta.true.linear + .6 * simulated.data$C + error

df <- cbind(Y, simulated.data)
colnames(df)[1] <- 'Y'

X.colnames <- colnames(X)

## Use LASSO of Y on X to select H

In [13]:
lasso.fit.outcome <- cv.glmnet(data.matrix(X), df$Y, alpha=1)

coef <- predict(lasso.fit.outcome, type = "nonzero")
H <- X.colnames[unlist(coef)]
# Variables selected by LASSO:
H

[1] "V2"    "V3"    "V4"    "V5"    "V6"    "V8"    "V9"    "V10"   "V11"  
[10] "V12"   "V13"   "V14"   "V15"   "V16"   "V18"   "V19"   "V20"   "V21"  
[19] "V22"   "V23"   "V24"   "V25"   "V26"   "V27"   "V28"   "V29"   "V30"  
[28] "V80"   "V81"   "V304"  "V745"  "V1049"

## Use LASSO of W on X to select K

In [14]:
lasso.fit.propensity <- cv.glmnet(data.matrix(X), df$W, alpha=1)

coef <- predict(lasso.fit.propensity, type = "nonzero")
K <- X.colnames[unlist(coef)]
# Variables selected by LASSO:
K

[1] "V2"    "V80"   "V81"   "V1049"

## Perform RDD of Y on W, Controlling for $H \cup K$

In [15]:
# Union of selected variables:
H_union_K.names <- unique(c(H, K))
H_union_K.names
sum.H_union_K <- paste(H_union_K.names, collapse = " + ")
eq.H_union_K <- paste("Y ~ W | ", sum.H_union_K)

# RDD, using all covariates selected by double selection
fit <- RDestimate(eq.H_union_K, data = df)
summary(fit)

[1] "V2"    "V3"    "V4"    "V5"    "V6"    "V8"    "V9"    "V10"   "V11"  
[10] "V12"   "V13"   "V14"   "V15"   "V16"   "V18"   "V19"   "V20"   "V21"  
[19] "V22"   "V23"   "V24"   "V25"   "V26"   "V27"   "V28"   "V29"   "V30"  
[28] "V80"   "V81"   "V304"  "V745"  "V1049"


Call:
RDestimate(formula = eq.H_union_K, data = df)

Type:
sharp 

Estimates:
           Bandwidth  Observations  Estimate  Std. Error  z value  Pr(>|z|) 
LATE       2.081      345           1.422     0.2914      4.881    1.054e-06
Half-BW    1.041      180           1.636     0.4205      3.891    9.963e-05
Double-BW  4.163      628           1.246     0.2196      5.673    1.406e-08
              
LATE       ***
Half-BW    ***
Double-BW  ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

F-statistics:
           F      Num. DoF  Denom. DoF  p
LATE       18314  35        309         0
Half-BW     9944  35        144         0
Double-BW  35587  35        592         0
